In [19]:
import numpy as np
from Encapsulation import *
from Linear import *
from NonLinear import *
from Loss import *
from convolution import *
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [20]:
def load_usps(fn):
    with open(fn,"r") as f:
        f.readline()
        data = [[float(x) for x in l.split()] for l in f if len(l.split())>2]
    tmp=np.array(data)
    return tmp[:,1:],tmp[:,0].astype(int)


X_train, y_train = load_usps("data/USPS_train.txt")
X_test, y_test = load_usps("data/USPS_test.txt")
y_train = OneHotEncoder().fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = OneHotEncoder().fit_transform(y_test.reshape(-1, 1)).toarray()

In [21]:
print(X_train.shape)
print(y_train.shape)

(7291, 256)
(7291, 10)


In [22]:
def normalize(data_x):
    x_min=np.min(data_x)
    x_max=np.max(data_x)
    return (data_x-x_min)/(1.0*(x_max-x_min))

In [23]:
X_train_norm=normalize(X_train)
X_test_norm=normalize(X_test)


In [24]:
liste_coder=[Conv1D(3,1,32,1),MaxPool1D(2,2),Flatten(),Linear(4064,100),ReLU(),Linear(100,10)]

network_usps=Sequential(liste_coder)
optim_usps=Optim(network_usps,CrossEntropyLoss(),eps=0.001)
results_df=optim_usps.SGD_eval(X_train_norm[:, :, np.newaxis],y_train,128,25)

Batch:   0%|          | 0/51 [00:02<?, ?it/s]


In [25]:
print("Le test score est : ",optim_usps.score(X_test[:, :, np.newaxis], y_test))
print("Le train score est : ",optim_usps.score(X_train[:, :, np.newaxis], y_train))

Le test score est :  0.9327354260089686
Le train score est :  0.9809353998079824
